# Environment setup

Launching this artifact on trovi has already provisioned a single baremetal node for us, and installed this Jupyter environment. Now, we'll need to finish the setup by installing our experiment's runtime requirements.

This simple power measurement experiment will use the `perf` program to query the CPU's power registers. For more information in general about power management, see [this blog post](https://chameleoncloud.org/blog/2024/06/18/power-measurement-and-management-on-chameleon/).

The setup script will update apt packages, and install the following:
- `stress-ng` - to run a stress test process on the CPU
- `linux-tools` - which contains the `perf` binary allowing us to read the power registers

Then the setup script load the Intel RAPL kernel module `intel_rapl_msr`.

In [ ]:
import os

os.system("scripts/setup.sh")

# Experiment execution

Now, we can finally run the experiment. This will run the `stress-ng` program on different numbers of CPUs for 10 seconds, and measures the power consumption via `perf`.

The `run.sh` script handles running these commands and capturing the output into the `./out` directory.

The following section of code runs this script several times. You can change the number of iterations to gather more data points, which may result in a more interesting result.

In [ ]:
iterations = 3
for i in range(iterations):
    print(f"Iteration {i+1}/{iterations}:")
    os.system("bash scripts/run.sh 10")
    print()
print("Done!")

# Analyze results

Now we can load the data with python into our notebook, and then plot it using `matplotlib`

In [ ]:
from collections import defaultdict
data = {
    "power/energy-pkg/": defaultdict(list),
    "power/energy-ram/": defaultdict(list),
}

for filename in os.listdir("out/"):
    # Only parse our data files
    if ".out" not in filename:
        continue
    with open(f"out/{filename}") as f:
        for line in f.readlines():
            line = line.strip()
            cores, value, measurement = line.split(" ")
            data[measurement][cores].append(float(value))


In [ ]:
import matplotlib.pyplot as plt

# Format perf's label into a nicer string
PERF_CHART_TYPE_FORMAT = {
    "power/energy-pkg/": "CPU",
    "power/energy-ram/": "RAM",
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
axes = iter([ax1, ax2])

for chart_type, chart_data in data.items():
    subplot = next(axes)
    labels = list(chart_data.keys())
    values = list(chart_data.values())
    subplot.boxplot(values, tick_labels=labels)
    
    subplot.set_title(f'{PERF_CHART_TYPE_FORMAT[chart_type]} Energy Consumption\nfor CPU Utilization % Box Plot')
    subplot.set_xlabel('CPU Utilization %')
    subplot.set_ylabel('Joules')
plt.show()